In [1]:
from needed_files.quantumenvironment import QuantumEnvironment
from needed_files.helper_functions import load_q_env_from_yaml_file, load_agent_from_yaml_file
from needed_files.ppo import make_train_ppo
from needed_files.q_env_config import q_env_config as gate_q_env_config

import time

/Users/lukasvoss/anaconda3/envs/braket/lib/python3.10/site-packages/qiskit_dynamics/dispatch/backends/jax.py:34: UserWarning: The functionality in the perturbation module of Qiskit Dynamics requires a JAX version <= 0.4.6, due to a bug in JAX versions > 0.4.6. For versions 0.4.4, 0.4.5, and 0.4.6, using the perturbation module functionality requires setting os.environ['JAX_JIT_PJIT_API_MERGE'] = '0' before importing JAX or Dynamics.
  warnings.warn(


ModuleNotFoundError: No module named 'helper_functions'

In [ ]:
from braket.tracking import Tracker
from braket.jobs import hybrid_job, save_job_result
from braket.jobs.metrics import log_metric
from braket.aws import AwsDevice

from braket.aws import AwsSession
from braket.aws import AwsQuantumJob
from braket.jobs.local import LocalQuantumJob

from qiskit_braket_provider import BraketLocalBackend, AWSBraketProvider, AWSBraketBackend, AWSBraketJob

aws_session = AwsSession(default_bucket="amazon-braket-us-west-1-lukasvoss")

In [ ]:
def calibrate_gate(num_total_updates):
    print("Job started!!!!!")

    braket_task_costs = Tracker().start()
    ppo_params, network_params  = load_agent_from_yaml_file(file_path='/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control/template_configurations/agent_config.yaml')
    agent_config = {**ppo_params, **network_params}

    
    provider = AWSBraketProvider()
    backend = provider.get_backend('SV1')
    q_env = QuantumEnvironment(gate_q_env_config)
    q_env.backend = backend
    
    ppo_agent = make_train_ppo(agent_config, q_env)

    # training_results = ppo_agent(total_updates=hyperparams["num_updates"], print_debug=True, num_prints=40)
    training_results = ppo_agent(total_updates=num_total_updates, print_debug=False, num_prints=40)
    # save_job_result({ "measurement_counts": training_results })

    print("Job completed!!!!!")

    training_results['task_summary'] = braket_task_costs.quantum_tasks_statistics()
    training_results['estimated cost'] = float(
            braket_task_costs.qpu_tasks_cost() + braket_task_costs.simulator_tasks_cost()
        )

    return training_results

In [ ]:
%%time

calibrate_gate(num_total_updates=1)